<b>Utis for data retrieving and Preprocessing</b>

In [7]:
from sklearn.model_selection import train_test_split

def get_data_partition(start, end, test_size):
    ids = []
    for idx in range(start, end):
        ids.append('rain_mat_' + str(idx))
    train, test = train_test_split(ids, test_size=test_size)
    return {'train': train, 'validation': test}


<b>Datagenerator class for retrieving training data on the fly</b>

In [8]:
import numpy as np
from keras.utils import Sequence
from os import path

class DataGenerator(Sequence):
    def __init__(self, list_ids, batch_size=32, dim=(32, 32, 32), reshape_dim=None, shuffle=True):
        """Initialization"""
        self.dim = dim
        self.reshape_dim = reshape_dim
        self.batch_size = batch_size
        self.list_ids = list_ids
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.list_ids) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_ids_temp = [self.list_ids[k] for k in indexes]

        # Generate data
        X, y = self.data_generation(list_ids_temp)

        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.list_ids))
        if self.shuffle is True:
            np.random.shuffle(self.indexes)

    def data_generation(self, list_ids_temp):
        """Generates data containing batch_size samples"""
        # Initialization
        X = np.empty((self.batch_size, np.prod(self.dim)))

        # Generate data
        for i, _id in enumerate(list_ids_temp):
            # Store sample
            data = np.load(path.join('/home/nira/PycharmProjects/DI_Framework/rain-matrix-3d/matrix_out', _id))
            X[i, ] = np.reshape(data, np.prod(self.dim))
        return X, X


<b>Import section for Neural Network modelling and Training</b>

In [9]:
import numpy as np
import pandas as pd
import sklearn

from keras import backend as T
from keras.layers import Activation, Input, Dense
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

<b>Rainfall retrieval and data preprocessing</b>

In [10]:
x_length = 100
y_length = 60
tms_length = 24

img_shape = (tms_length*x_length*y_length,)
l1_out_units = x_length*y_length
l2_out_units = int(np.sqrt(np.square(x_length) + np.square(y_length)))

In [11]:
input_matrix = Input(shape=img_shape)

encode_l1 = Dense(units=l1_out_units, activation='relu')(input_matrix)
encode_l2 = Dense(units=l2_out_units, activation='relu')(encode_l1)

decode_l1 = Dense(units=l1_out_units, activation='relu')(encode_l2)
decode_l2 = Dense(units=img_shape[0], activation='softmax')(decode_l1)

print(img_shape[0])

autoencoder = Model(input_matrix, decode_l2)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

144000


In [12]:
params = {'dim': (tms_length, x_length, y_length), 'batch_size': 3, 'shuffle': True}
partition = get_data_partition(1, 12, 0.2)

training_generator = DataGenerator(partition['train'], **params)
validation_generator = DataGenerator(partition['validation'], **params)

In [13]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 144000)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 6000)              864006000 
_________________________________________________________________
dense_6 (Dense)              (None, 116)               696116    
_________________________________________________________________
dense_7 (Dense)              (None, 6000)              702000    
_________________________________________________________________
dense_8 (Dense)              (None, 144000)            864144000 
Total params: 1,729,548,116
Trainable params: 1,729,548,116
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder.fit_generator(generator=training_generator,
                          validation_data=validation_generator,
                          use_multiprocessing=True,
                          workers=2,
                          epochs=4)

Epoch 1/4


In [11]:
model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 16)                32        
_________________________________________________________________
dense_18 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer=Adam(lr=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])